# Time Series Anomaly Detection with aeon

The aim of Time Series Anomaly Detection is to discover regions of a time series that in some way are not representative of the underlying generative process. An anomaly in time series can be a single point or a subsequence that deviates from the regular patterns of the sequence with respect to some measure, model or embedding. This notebook gives an overview of anomaly detection module and the available detectors.

<img src="img/anomaly_detection.png" width="600" alt="time series anomalies">

## Data Storage and Problem types

The anomaly detectors implemented in `aeon.anomaly_detection` module have different capabilities and can be grouped according to the following categories.

<img src="img/detector_classification.png" width=600 alt="taxonomy of outlier detectors in aeon">

## Anomaly Detection in aeon

In [2]:
from aeon.registry import all_estimators

all_estimators("anomaly-detector", as_dataframe=True)

,name,estimator
0,DWT_MLEAD,<class 'aeon.anomaly_detection._dwt_mlead.DWT_...
1,KMeansAD,<class 'aeon.anomaly_detection._kmeans.KMeansAD'>
2,MERLIN,<class 'aeon.anomaly_detection._merlin.MERLIN'>
3,PyODAdapter,<class 'aeon.anomaly_detection._pyodadapter.Py...
4,STOMP,<class 'aeon.anomaly_detection._stomp.STOMP'>
5,STRAY,<class 'aeon.anomaly_detection._stray.STRAY'>
